# Index

- **Python Libraries**
- **Task 2**
    - *Reading Preprocessed Datasets*
    - *Exercise 1*
    - *Exercise 2*
        - *Balancing Strategy Definition*
        - *Exercise 2.1*
        - *Exercise 2.2*
        - *Exercise 2.3*
    - *Exercise 3 - Concentration*
    - *Writing Balanced Datasets*


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import random

# Task 2

## Reading Preprocessed Datasets

In [ ]:
read_data_path  = "../../data/dataset_preprocessed"

ap   = pd.read_csv(read_data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso','label'])
diag = pd.read_csv(read_data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(read_data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei   = pd.read_csv(read_data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf  = pd.read_csv(read_data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(read_data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd  = pd.read_csv(read_data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [ ]:
print(ap.shape)
ap.head()

(2242, 13)


,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,M,1986.0,5,2.0,2.0,9.0,NaN,1942,1990.0,2014.0,1
1,1,46,F,1982.0,5,2.0,2.0,4.0,NaN,1937,1997.0,2016.0,1
2,1,60,F,1987.0,5,2.0,2.0,9.0,NaN,1923,1987.0,2010.0,0
3,1,65,M,1997.0,5,2.0,1.0,9.0,NaN,1942,1997.0,2018.0,1
4,1,74,M,1995.0,5,2.0,2.0,9.0,NaN,1930,1996.0,2013.0,0


In [ ]:
print(diag.shape)
diag.head()

(155217, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,1997-12-01,AMD247,410
1,1,5,2000-06-01,AMD247,434.01
2,1,5,2000-06-01,AMD247,434.91
3,1,5,2000-12-02,AMD049,S
4,1,5,2000-12-02,AMD247,36.10


In [ ]:
print(elp.shape)
elp.head()

(451271, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.000000
1,1,5,2005-01-18,AMD002,76.000000
2,1,5,2005-01-18,AMD004,90.232558
3,1,5,2005-01-18,AMD005,47.346939
4,1,5,2005-06-06,AMD007,138.225058


In [ ]:
print(ei.shape)
ei.head()

(21652, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [ ]:
print(pdf.shape)
pdf.head()

(134030, 7)


,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,1,5,2005-01-18,A10BA02,2.0,1.0,METFORAL*50CPR RIV 500mg
1,1,5,2005-01-18,A10BA02,2.0,3.0,METFORAL*50CPR RIV 500mg
2,1,5,2005-01-18,A10BA02,2.0,5.0,METFORAL*50CPR RIV 500mg
3,1,5,2005-01-18,A10BB12,1.0,3.0,AMARYL*30CPR 2mg
4,1,5,2005-06-21,A10BA02,2.0,1.0,METFORAL*50CPR RIV 500mg


In [ ]:
print(pdnf.shape)
pdnf.head()

(14071, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,46,2007-01-26,AMD152,4015630065


In [ ]:
print(pnd.shape)
pnd.head()

(147973, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


## Exercise 1

*Class imbalance #1* - not all patients will have a cardiovascular event within the stabilised six-month period. Thus, we would expect that the class distribution is highly imbalanced. 

- For each patient $p_i$ such that $y(p_i)=0$, eliminate the last six months of history to avoid giving the model prediction hints into the future. 
- For each patient $p_i$ such that $y(p_i)=1$, create $m$ copies $\lbrace p_i^1, \cdots, p_i^m \rbrace$ such that all the cardiovascular events in the last six months are eliminated, and the other events are shuffled and cancelled at random.

In this way, you have a sort of balancing criterion (i.e., up-sampling the minority class).

First we divide the **ap** dataframe into:
- **ap_class0** each patient $p_i$ has $y(p_i)=0$
- **ap_class1** each patient $p_i$ has $y(p_i)=1$

In [ ]:
print(f"ap has size        => {ap.shape}")
ap_class0 = ap[ap.label == 0]
print(f"ap_class0 has size => {ap_class0.shape}")
ap_class1 = ap[ap.label == 1]
print(f"ap_class1 has size => {ap_class1.shape}")


ap has size        => (2242, 13)
ap_class0 has size => (1221, 13)
ap_class1 has size => (1021, 13)


And now the rest of the dataframes are also divided into **df_class0** and **df_class1** following the same criteria:

In [ ]:
diag_class0 = pd.merge(diag, ap_class0, on=['idcentro','idana'])[diag.columns]
elp_class0  = pd.merge(elp, ap_class0, on=['idcentro','idana'])[elp.columns]
ei_class0   = pd.merge(ei, ap_class0, on=['idcentro','idana'])[ei.columns]
pdf_class0  = pd.merge(pdf, ap_class0, on=['idcentro','idana'])[pdf.columns]
pdnf_class0 = pd.merge(pdnf, ap_class0, on=['idcentro','idana'])[pdnf.columns]
pnd_class0  = pd.merge(pnd, ap_class0, on=['idcentro','idana'])[pnd.columns]

diag_class1 = pd.merge(diag, ap_class1, on=['idcentro','idana'])[diag.columns]
elp_class1  = pd.merge(elp, ap_class1, on=['idcentro','idana'])[elp.columns]
ei_class1   = pd.merge(ei, ap_class1, on=['idcentro','idana'])[ei.columns]
pdf_class1  = pd.merge(pdf, ap_class1, on=['idcentro','idana'])[pdf.columns]
pdnf_class1 = pd.merge(pdnf, ap_class1, on=['idcentro','idana'])[pdnf.columns]
pnd_class1  = pd.merge(pnd, ap_class1, on=['idcentro','idana'])[pnd.columns]

We define a function that eliminates the events during the last 6 months for each patient

In [ ]:
def dropLastSixMonths(df:pd.DataFrame, patients: pd.DataFrame) -> pd.DataFrame:

    newDF = pd.DataFrame(columns=df.columns)

    # We iterate for each patient on patients
    for p in patients.itertuples():
        # Take the cardiovascular events of the patient p
        aux = df[(df.idcentro==p.idcentro) & (df.idana==p.idana)]

        # Calculate the last valid data
        aux = aux.sort_values('data', ascending=False)
        lastData = next(aux.itertuples()).data
        lastData = dt.date.fromisoformat(lastData)
        limitData = lastData - dt.timedelta(days=30*6)

        # Take only the events before this limit data
        aux = aux[aux.data < limitData.strftime("%Y-%m-%d")]
        
        # We concatenate this with the new DataFrame
        newDF = pd.concat([newDF, aux])

    return newDF

Now we execute the function to every dataframe in our preprocessed dataset for the **ap_class0** patients:

In [ ]:
aux = diag_class0.shape[0]
diag_class0 = dropLastSixMonths(diag_class0, ap_class0)
print(f"Diag_class0: {aux} => {diag_class0.shape[0]}")

aux = elp_class0.shape[0]
elp_class0 = dropLastSixMonths(elp_class0, ap_class0)
print(f"Elp_class0: {aux} => {elp_class0.shape[0]}")

aux = ei_class0.shape[0]
ei_class0 = dropLastSixMonths(ei_class0, ap_class0)
print(f"Ei_class0: {aux} => {ei_class0.shape[0]}")

aux = pdf_class0.shape[0]
pdf_class0 = dropLastSixMonths(pdf_class0, ap_class0)
print(f"Pdf_class0: {aux} => {pdf_class0.shape[0]}")

aux = pdnf_class0.shape[0]
pdnf_class0 = dropLastSixMonths(pdnf_class0, ap_class0)
print(f"Pdnf_class0: {aux} => {pdnf_class0.shape[0]}")

aux = pnd_class0.shape[0]
pnd_class0 = dropLastSixMonths(pnd_class0, ap_class0)
print(f"Pnd_class0: {aux} => {pnd_class0.shape[0]}")

Diag_class0: 68853 => 60598
Elp_class0: 234208 => 203837
Ei_class0: 10714 => 8519
Pdf_class0: 69059 => 61643
Pdnf_class0: 7983 => 6003
Pnd_class0: 71844 => 63004


Now for the **ap_class1** patients:

In [ ]:
aux = diag_class1.shape[0]
diag_class1 = dropLastSixMonths(diag_class1, ap_class1)
print(f"Diag_class1: {aux} => {diag_class1.shape[0]}")

aux = elp_class1.shape[0]
elp_class1 = dropLastSixMonths(elp_class1, ap_class1)
print(f"Elp_class1: {aux} => {elp_class1.shape[0]}")

aux = ei_class1.shape[0]
ei_class1 = dropLastSixMonths(ei_class1, ap_class1)
print(f"Ei_class1: {aux} => {ei_class1.shape[0]}")

aux = pdf_class1.shape[0]
pdf_class1 = dropLastSixMonths(pdf_class1, ap_class1)
print(f"Pdf_class1: {aux} => {pdf_class1.shape[0]}")

aux = pdnf_class1.shape[0]
pdnf_class1 = dropLastSixMonths(pdnf_class1, ap_class1)
print(f"Pdnf_class1: {aux} => {pdnf_class1.shape[0]}")

aux = pnd_class1.shape[0]
pnd_class1 = dropLastSixMonths(pnd_class1, ap_class1)
print(f"Pnd_class1: {aux} => {pnd_class1.shape[0]}")

Diag_class1: 86364 => 73887
Elp_class1: 217063 => 189759
Ei_class1: 10938 => 8790
Pdf_class1: 64971 => 57872
Pdnf_class1: 6088 => 4478
Pnd_class1: 76129 => 67051


At this point, we are done with the **ap_class0** patients, so we focus on the **ap_class1** ones. We now have to create $m$ (= 10 for example) copies of each patient $p_i$ such that $y(p_i)=1$, and we shuffle or eliminate cardiovascular events randomly just to *oversample* the dataset:

In [ ]:
def newMCopies(patients:pd.DataFrame, copies:int) -> pd.DataFrame:

    cols = patients.columns.tolist()
    cols = cols[0:2]+['idcopy']+cols[2:]
    newPatients = pd.DataFrame(columns=cols)

    for p in patients.iterrows():
        for i in range(copies):
            newP = dict(p[1])
            newP['idcopy'] = i+1
            newPatients.loc[newPatients.shape[0]] = newP

    return newPatients

In [ ]:
M = 3
ap_class1_copies = newMCopies(ap_class1, M)

We can check that every patient has been cloned M times, and the only identification for each clone $p_i^j$ is $i=\lbrace$ idcentro, idana $\rbrace$ and $j = \lbrace$ idcopy $\rbrace$

In [ ]:
print("Patient 1")
ap_class1_copies.iloc[0:1*M]

Patient 1


,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,1,M,1986.0,5,2.0,2.0,9.0,NaN,1942,1990.0,2014.0,1
1,1,5,2,M,1986.0,5,2.0,2.0,9.0,NaN,1942,1990.0,2014.0,1
2,1,5,3,M,1986.0,5,2.0,2.0,9.0,NaN,1942,1990.0,2014.0,1


In [ ]:
print("Patient 2")
ap_class1_copies.iloc[M:2*M]

Patient 2


,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso,label
3,1,46,1,F,1982.0,5,2.0,2.0,4.0,NaN,1937,1997.0,2016.0,1
4,1,46,2,F,1982.0,5,2.0,2.0,4.0,NaN,1937,1997.0,2016.0,1
5,1,46,3,F,1982.0,5,2.0,2.0,4.0,NaN,1937,1997.0,2016.0,1


In [ ]:
def newMCopiesRandom(df:pd.DataFrame, copies:int) -> pd.DataFrame:

    # We add the column idcopy to our table
    cols = df.columns.tolist()
    cols = cols[0:2]+['idcopy']+cols[2:]
    newDf = pd.DataFrame(columns=cols)

    # For each event we make copies shuffling and cancelling randomly
    for p in df.iterrows():
        z = random.randrange(100)
        for i in range(copies):
          y = random.randrange(100)
          if (y%3!= 0):
            newEvent = dict(p[1])
            newEvent['idcopy'] = z%copies + 1
            newDf.loc[newDf.shape[0]] = newEvent
          z = z+1

    return newDf

In [ ]:
M = 3
diag_class1_copies = newMCopiesRandom(diag_class1, M)

In [ ]:
print("Patient 1")
diag_class1_copies.iloc[0:50]

Patient 1


,idcentro,idana,idcopy,data,codiceamd,valore
0,1,5,2,2013-06-28,AMD247,37.94
1,1,5,3,2013-06-28,AMD247,37.94
2,1,5,1,2013-06-28,AMD247,37.94
3,1,5,2,2013-06-28,AMD247,36.10
4,1,5,1,2013-06-28,AMD247,36.10
5,1,5,2,2013-06-28,AMD049,S
6,1,5,1,2013-06-28,AMD247,401
7,1,5,3,2013-06-28,AMD247,402
8,1,5,1,2013-06-28,AMD247,410
9,1,5,3,2013-06-28,AMD247,428.0


In [ ]:
elp_class1_copies = newMCopiesRandom(df=elp_class1, copies=M)
ei_class1_copies = newMCopiesRandom(df=ei_class1, copies=M)
pdf_class1_copies = newMCopiesRandom(df=pdf_class1, copies=M)
pdnf_class1_copies = newMCopiesRandom(df=pdnf_class1, copies=M)
pnd_class1_copies = newMCopiesRandom(df=pnd_class1, copies=M)

KeyboardInterrupt: ignored

## Exercise 2

*Class Imbalance #2* - Action item *#1* isn't going to be sufficient for balancing purposes. Propose your balancing strategy - possibly an advanced approach - and evaluate: 
- **vanilla-LTSM** (*Exercise 2.1*)
- **T-LTSM** (*Exercise 2.2*)
- **PubMedBert** (*Exercise 2.3*) 

on the balanced version of the dataset.

### Balancing Strategy Definition

### Exercise 2.1

### Exercise 2.2

### Exercise 2.3